# Scaping NFL data


In [157]:
# Importing libraries
import random
import time

import numpy as np
import pandas as pd

In [158]:
# Create list of seasons as stings to scrape
seasons = [str(season) for season in range(2015, 2025)]
print(f"Scraping data for {len(seasons)} seasons")

# Create list of team abbreviations used by pro-football-reference
teams = [
    "crd",
    "atl",
    "rav",
    "buf",
    "car",
    "chi",
    "cin",
    "cle",
    "dal",
    "den",
    "det",
    "gnb",
    "htx",
    "clt",
    "jax",
    "kan",
    "rai",
    "sdg",
    "ram",
    "mia",
    "min",
    "nwe",
    "nor",
    "nyg",
    "nyj",
    "phi",
    "pit",
    "sfo",
    "sea",
    "tam",
    "oti",
    "was",
]
print(f"Scraping data for {len(teams)} teams")

Scraping data for 10 seasons
Scraping data for 32 teams


In [159]:
# Empty list to store dataframes
all_dfs = []

# Loop through each season for each team
for season in seasons:
    for team in teams:
        # 1. Create url
        url = f"https://www.pro-football-reference.com/teams/{team}/{season}/gamelog/"
        print(url)

        # 2. Scrape data
        df = pd.read_html(url, header=1, attrs={"id": "gamelog" + season})[0]

        # 3. Insert Team and Season into dataframe
        df.insert(loc=0, column="Season", value=season)
        df.insert(loc=2, column="Team", value=team.upper())

        # 4. Append to list
        all_dfs.append(df)

        # Limit requests to 20 requests per minute
        time.sleep(random.randint(3, 5))

# Concatenate all dataframes in list
gamelog_df = pd.concat(all_dfs, ignore_index=True)
gamelog_df.to_csv(f"../data/nfl_gamelog_{seasons[0]}-{seasons[-1]}.csv", index=False)

https://www.pro-football-reference.com/teams/crd/2015/gamelog/
https://www.pro-football-reference.com/teams/atl/2015/gamelog/
https://www.pro-football-reference.com/teams/rav/2015/gamelog/
https://www.pro-football-reference.com/teams/buf/2015/gamelog/
https://www.pro-football-reference.com/teams/car/2015/gamelog/
https://www.pro-football-reference.com/teams/chi/2015/gamelog/
https://www.pro-football-reference.com/teams/cin/2015/gamelog/
https://www.pro-football-reference.com/teams/cle/2015/gamelog/
https://www.pro-football-reference.com/teams/dal/2015/gamelog/
https://www.pro-football-reference.com/teams/den/2015/gamelog/
https://www.pro-football-reference.com/teams/det/2015/gamelog/
https://www.pro-football-reference.com/teams/gnb/2015/gamelog/
https://www.pro-football-reference.com/teams/htx/2015/gamelog/
https://www.pro-football-reference.com/teams/clt/2015/gamelog/
https://www.pro-football-reference.com/teams/jax/2015/gamelog/
https://www.pro-football-reference.com/teams/kan/2015/g

In [160]:
np.set_printoptions(linewidth=120)

gamelog_df = pd.read_csv(f"../data/nfl_gamelog_{seasons[0]}-{seasons[-1]}.csv")
print(gamelog_df.info())
print(gamelog_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5246 entries, 0 to 5245
Data columns (total 38 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Season      5246 non-null   int64  
 1   Week        5246 non-null   int64  
 2   Team        5246 non-null   object 
 3   Day         5246 non-null   object 
 4   Date        5246 non-null   object 
 5   Unnamed: 3  5246 non-null   object 
 6   Unnamed: 4  5246 non-null   object 
 7   OT          304 non-null    object 
 8   Unnamed: 6  2623 non-null   object 
 9   Opp         5246 non-null   object 
 10  Tm          5246 non-null   int64  
 11  Opp.1       5246 non-null   int64  
 12  Cmp         5246 non-null   int64  
 13  Att         5246 non-null   int64  
 14  Yds         5246 non-null   int64  
 15  TD          5246 non-null   int64  
 16  Int         5246 non-null   int64  
 17  Sk          5246 non-null   int64  
 18  Yds.1       5246 non-null   int64  
 19  Y/A         5246 non-null  

## Clean Gamelog Data


In [161]:
# Drop Game stats columns
gamelog_clean_df = gamelog_df.drop(gamelog_df.columns[12:], axis=1)
# Drop Boxscore Link column
gamelog_clean_df = gamelog_clean_df.drop(["Unnamed: 3"], axis=1)
# Drop 'Day' column
gamelog_clean_df = gamelog_clean_df.drop(["Day"], axis=1)
# Drop 'OT' column
gamelog_clean_df = gamelog_clean_df.drop(["OT"], axis=1)

# Rename columns
gamelog_clean_df = gamelog_clean_df.rename(
    columns={
        "Unnamed: 4": "Win",
        "Unnamed: 6": "Home",
        "Tm": "Team_Pts",
        "Opp.1": "Opp_Pts",
    }
)

# Map Opp to team abbreviation
# Current team names to abbreviations
current_team_dict = {
    "Arizona Cardinals": "CRD",
    "Atlanta Falcons": "ATL",
    "Baltimore Ravens": "RAV",
    "Buffalo Bills": "BUF",
    "Carolina Panthers": "CAR",
    "Chicago Bears": "CHI",
    "Cincinnati Bengals": "CIN",
    "Cleveland Browns": "CLE",
    "Dallas Cowboys": "DAL",
    "Denver Broncos": "DEN",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GNB",
    "Houston Texans": "HTX",
    "Indianapolis Colts": "CLT",
    "Jacksonville Jaguars": "JAX",
    "Kansas City Chiefs": "KAN",
    "Los Angeles Chargers": "SDG",
    "Los Angeles Rams": "RAM",
    "Las Vegas Raiders": "RAI",
    "Miami Dolphins": "MIA",
    "Minnesota Vikings": "MIN",
    "New England Patriots": "NWE",
    "New Orleans Saints": "NOR",
    "New York Giants": "NYG",
    "New York Jets": "NYJ",
    "Philadelphia Eagles": "PHI",
    "Pittsburgh Steelers": "PIT",
    "San Francisco 49ers": "SFO",
    "Seattle Seahawks": "SEA",
    "Tampa Bay Buccaneers": "TAM",
    "Tennessee Titans": "OTI",
    "Washington Commanders": "WAS",
}

# Historical/alternative names
historical_team_dict = {
    "Oakland Raiders": "RAI",
    "Washington Football Team": "WAS",
    "Washington Redskins": "WAS",
}
full_team_dict = {**current_team_dict, **historical_team_dict}
gamelog_clean_df["Opp"] = gamelog_clean_df["Opp"].map(full_team_dict)

# Convert 'Win' column to boolean (W = 1, L = 0)
gamelog_clean_df["Win"] = gamelog_clean_df["Win"].apply(lambda x: 1 if x == "W" else 0)
# Convert 'Home' column to boolean (H = 1, A = 0)
gamelog_clean_df["Home"] = gamelog_clean_df["Home"].apply(
    lambda x: 0 if x == "@" else 1
)

with pd.option_context("display.max_colwidth", None, "display.max_columns", None):
    display(gamelog_clean_df)

,Season,Week,Team,Date,Win,Home,Opp,Team_Pts,Opp_Pts
0,2015,1,CRD,September 13,1,1,NOR,31,19
1,2015,2,CRD,September 20,1,0,CHI,48,23
2,2015,3,CRD,September 27,1,1,SFO,47,7
3,2015,4,CRD,October 4,0,1,NaN,22,24
4,2015,5,CRD,October 11,1,0,DET,42,17
...,...,...,...,...,...,...,...,...,...
5241,2024,13,WAS,December 1,1,1,OTI,42,19
5242,2024,15,WAS,December 15,1,0,NOR,20,19
5243,2024,16,WAS,December 22,1,1,PHI,36,33
5244,2024,17,WAS,December 29,1,1,ATL,30,24


## Scrape vegas lines data


In [162]:
# Create empty list to store dataframes
all_lines_dfs = []

# Loop through each season for each team
for season in seasons:
    for team in teams:
        # 1. Create url
        url = f"https://www.pro-football-reference.com/teams/{team}/{season}_lines.htm"
        print(url)

        # 2. Scrape data
        lines_df = pd.read_html(url, header=0, attrs={"id": "vegas_lines"})[0]

        # 3. Insert Team and Season into dataframe
        lines_df.insert(loc=0, column="Season", value=season)
        lines_df.insert(loc=2, column="Team", value=team.upper())

        # 4. Append to list
        all_lines_dfs.append(lines_df)

        # Limit requests to 20 requests per minute
        time.sleep(random.randint(3, 5))

# Concatenate all dataframes in list
lines_df = pd.concat(all_lines_dfs, ignore_index=True)
lines_df.to_csv(f"../data/nfl_lines_{seasons[0]}-{seasons[-1]}.csv", index=False)

https://www.pro-football-reference.com/teams/crd/2015_lines.htm
https://www.pro-football-reference.com/teams/atl/2015_lines.htm
https://www.pro-football-reference.com/teams/rav/2015_lines.htm
https://www.pro-football-reference.com/teams/buf/2015_lines.htm
https://www.pro-football-reference.com/teams/car/2015_lines.htm
https://www.pro-football-reference.com/teams/chi/2015_lines.htm
https://www.pro-football-reference.com/teams/cin/2015_lines.htm
https://www.pro-football-reference.com/teams/cle/2015_lines.htm
https://www.pro-football-reference.com/teams/dal/2015_lines.htm
https://www.pro-football-reference.com/teams/den/2015_lines.htm
https://www.pro-football-reference.com/teams/det/2015_lines.htm
https://www.pro-football-reference.com/teams/gnb/2015_lines.htm
https://www.pro-football-reference.com/teams/htx/2015_lines.htm
https://www.pro-football-reference.com/teams/clt/2015_lines.htm
https://www.pro-football-reference.com/teams/jax/2015_lines.htm
https://www.pro-football-reference.com/t

In [163]:
lines_df = pd.read_csv(f"../data/nfl_lines_{seasons[0]}-{seasons[-1]}.csv")
print(lines_df.info())
print(lines_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484 entries, 0 to 5483
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Season      5484 non-null   int64  
 1   G#          5484 non-null   int64  
 2   Team        5484 non-null   object 
 3   Opp         5484 non-null   object 
 4   Spread      5484 non-null   float64
 5   Over/Under  5484 non-null   float64
 6   Result      5484 non-null   object 
 7   vs. Line    5484 non-null   object 
 8   OU Result   5484 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 385.7+ KB
None
(5484, 9)


## Clean Vegas Lines Data


In [164]:
# Drop columns
lines_clean_df = lines_df.drop(lines_df.columns[6:], axis=1)

# Rename columns
lines_clean_df = lines_clean_df.rename(
    columns={
        "G#": "G",
        "Over/Under": "Total",
    }
)

# Remove playoff games from dataframe
# 1978 - 2020 seasons had 16 regular season games per team
# 2021 - current(2025) seasons have 17 regular season games per team
lines_clean_df = lines_clean_df.query(
    "(Season <= 2020 & G < 17) | (Season >= 2021 & G < 18)"
)

# Remove 'G' column
lines_clean_df = lines_clean_df.drop(["G"], axis=1)

# Create 'Home' column (if opp has @ then it is away game)
lines_clean_df["Home"] = lines_clean_df["Opp"].apply(lambda x: 0 if x[0] == "@" else 1)

# Remove '@' from 'Opp' column
lines_clean_df["Opp"] = lines_clean_df["Opp"].str.replace("@", "")

# Map team abbreviations in vegas lines dataframe to team abbreviations in gamelog dataframe
abbr_dict = {
    "OAK": "RAI",
    "LVR": "RAI",
    "STL": "RAM",
    "LAR": "RAM",
    "LAC": "SDG",
    "IND": "CLT",
    "HOU": "HTX",
    "BAL": "RAV",
    "ARI": "CRD",
    "TEN": "OTI",
}
lines_clean_df = lines_clean_df.replace({"Opp": abbr_dict})

with pd.option_context("display.max_colwidth", None, "display.max_columns", None):
    display(lines_clean_df)

,Season,Team,Opp,Spread,Total,Home
0,2015,CRD,NOR,-2.0,48.5,1
1,2015,CRD,CHI,-2.0,46.0,0
2,2015,CRD,SFO,-6.5,45.0,1
3,2015,CRD,RAM,-7.0,44.0,1
4,2015,CRD,DET,-3.0,45.5,0
...,...,...,...,...,...,...
5476,2024,WAS,OTI,-6.0,44.5,1
5477,2024,WAS,NOR,-7.5,43.5,0
5478,2024,WAS,PHI,4.0,46.5,1
5479,2024,WAS,ATL,-3.5,46.5,1


## Merge Vegas Lines and Gamelog Data


In [165]:
# Check Both Dataframes
print(gamelog_clean_df.shape)
print(lines_clean_df.shape)

(5246, 9)
(5246, 6)


In [166]:
# Merge Dataframes using 'Season', 'Team', 'Opp', and 'Home' columns
merged_df = pd.merge(
    gamelog_clean_df, lines_clean_df, on=["Season", "Team", "Opp", "Home"]
)

# Check Merged Dataframe
print(merged_df.shape)
with pd.option_context("display.max_colwidth", None, "display.max_columns", None):
    display(merged_df)

(5198, 11)


,Season,Week,Team,Date,Win,Home,Opp,Team_Pts,Opp_Pts,Spread,Total
0,2015,1,CRD,September 13,1,1,NOR,31,19,-2.0,48.5
1,2015,2,CRD,September 20,1,0,CHI,48,23,-2.0,46.0
2,2015,3,CRD,September 27,1,1,SFO,47,7,-6.5,45.0
3,2015,5,CRD,October 11,1,0,DET,42,17,-3.0,45.5
4,2015,6,CRD,October 18,0,0,PIT,13,25,-3.5,45.0
...,...,...,...,...,...,...,...,...,...,...,...
5193,2024,13,WAS,December 1,1,1,OTI,42,19,-6.0,44.5
5194,2024,15,WAS,December 15,1,0,NOR,20,19,-7.5,43.5
5195,2024,16,WAS,December 22,1,1,PHI,36,33,4.0,46.5
5196,2024,17,WAS,December 29,1,1,ATL,30,24,-3.5,46.5


## Reformat Merged Dataframe


In [167]:
# Create clean version of merged dataframe
merged_clean_df = merged_df.copy()

# Change 'Team' column to use team names instead of abbreviations
reverse_current_dict = {v: k for k, v in current_team_dict.items()}
merged_clean_df["Team"] = (
    merged_clean_df["Team"].map(reverse_current_dict).str.split().str[-1]
)
merged_clean_df["Opp"] = (
    merged_clean_df["Opp"].map(reverse_current_dict).str.split().str[-1]
)

# Reformat 'Date' column to datetime
merged_clean_df["Date"] = pd.to_datetime(
    merged_clean_df["Date"], format="%B %d"
).dt.strftime("%d-%b")

with pd.option_context("display.max_colwidth", None, "display.max_columns", None):
    display(merged_clean_df)

,Season,Week,Team,Date,Win,Home,Opp,Team_Pts,Opp_Pts,Spread,Total
0,2015,1,Cardinals,13-Sep,1,1,Saints,31,19,-2.0,48.5
1,2015,2,Cardinals,20-Sep,1,0,Bears,48,23,-2.0,46.0
2,2015,3,Cardinals,27-Sep,1,1,49ers,47,7,-6.5,45.0
3,2015,5,Cardinals,11-Oct,1,0,Lions,42,17,-3.0,45.5
4,2015,6,Cardinals,18-Oct,0,0,Steelers,13,25,-3.5,45.0
...,...,...,...,...,...,...,...,...,...,...,...
5193,2024,13,Commanders,01-Dec,1,1,Titans,42,19,-6.0,44.5
5194,2024,15,Commanders,15-Dec,1,0,Saints,20,19,-7.5,43.5
5195,2024,16,Commanders,22-Dec,1,1,Eagles,36,33,4.0,46.5
5196,2024,17,Commanders,29-Dec,1,1,Falcons,30,24,-3.5,46.5


In [168]:
# Save Merged Dataframe
merged_clean_df.to_csv(
    f"../data/nfl_data_{seasons[0]}-{seasons[-1]}.csv", index=False
)
